# Comparsion of the EM and MM methods

## 1. Preloading packages and functions

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import cvxpy as cp
from scipy import optimize
from scripts.estimation import *
from scripts.utils import *

# 2. Generate Synthetic Data

We generate the synthetic data from a Poisson process with an underlying MNL model as rider's choice probability. In the MNL model, the intercept is `beta0` and the slope regarding the distance to an available bike is `beta1`. The arrival times of riders follow Poisson process with rate `lambd`. The number of generated arrival locations is set as `num_position`. The number of bikes that is included in the system is set as `bike_num`. We observe the process for in time period `[0,T]`. In this example, all arrival locations are generated randomly at an intersection of a 5x5 Cartesian grid.

In [2]:
rand_seed = 1
num_position = 10
bike_num = 30
lambd = 10
grid_size = 5
beta0 = 1
beta1_true = -1
T = 100
loc_bound = 5


In [3]:
true_pos_ind,bike_num,num_records,book_bike,book_index, \
dist,bike_loc,all_period,num_booked, \
cand_loc,true_loc,position_weight = gen_sync_in_grid(rand_seed,num_position,bike_num,lambd,
                                                     grid_size,beta0=beta0,beta1_true=beta1_true,T=T,loc_bound=loc_bound)

# 3. Implementation of the EM algorithm
The following snippet tests the performance of the EM algorithm. All arrival locations are randomly generated in a Cartesian grid that is chosen as the candidate location set for the all-in algorithm. To test the accuracy of the EM algorithm, we select the first `num_position` locations in the candidate set decreasing ordered by the predicted weights. We also compute the L-1 norm between the predicted weights and the underlying truth.

In [4]:
# Initialization of parameters
num_position_true = num_position
num_position = grid_size**2

w = np.random.uniform(0,1,num_position)
w = w/np.sum(w)
w = w.reshape(1,-1)
diff_w = np.inf
beta1 = np.repeat(-1,num_position).reshape(-1,1)
dist_old = caldist(cand_loc[-1],bike_loc,bike_num)
iter = 0
threshold = 1e-4

# EM iteration
while diff_w > threshold:
    iter = iter+1
    p_old = np.zeros((num_position,bike_num+1,num_records))
    p_olds = np.zeros((num_position,num_booked))
    w_old = w[-1,:]
    p_old[:,0,:] = 1/(1+np.sum(np.exp(beta0+np.reshape(beta1,(-1,1,1))*dist_old),axis=2))
    p_old[:,1:,:] = np.exp(beta0+np.reshape(beta1,(-1,1,1))*np.transpose(dist_old,(0,2,1)))/ \
        np.reshape((1+np.sum(np.exp(beta0+np.reshape(beta1,(-1,1,1))*dist_old),axis=2)),(num_position,1,-1))
    p_olds = p_old[:,book_bike+1,book_index]*np.reshape(w_old,(-1,1))/np.reshape(np.sum(np.reshape(w_old,(-1,1))*p_old[:,book_bike+1,book_index],axis=0),(1,-1))
    p_oldsj0 = w_old*np.sum(p_old[:,0,:]*all_period,axis=1)/np.sum(w_old*np.sum(p_old[:,0,:]*all_period,axis=1))
    s_old = np.sum((1-np.sum(np.expand_dims(w_old,1)*(1/(1+np.sum(np.exp(beta0+np.reshape(beta1,(-1,1,1))*dist_old),axis=2))),axis=0))*all_period)
    N_oldy = num_booked*(T-s_old)/s_old
    c = np.sum(p_olds,axis=1)+N_oldy*p_oldsj0
    w_star = np.reshape(c/np.sum(c),(1,-1))
    w = np.concatenate((w,w_star),axis=0)
    diff_w = np.sum(np.abs(w[-1,:]-w[-2,:]))

pre_w = w[-2]
true_w = np.zeros(grid_size**2)
true_w[true_pos_ind] = position_weight
sel_ind = np.where(pre_w>=np.sort(pre_w)[-num_position_true])[0]
true_pred_ind = np.intersect1d(sel_ind,true_pos_ind)
l1norm = np.sum(np.abs(pre_w-true_w))
lkd = findlkd_no_constraint(num_position,dist_old,
                                beta0,np.repeat(beta1_true,num_position).reshape(-1,1),pre_w,
                                bike_num,num_records,book_bike,book_index,num_booked,all_period)
print('Number of Iterations:', iter)
print('Accuracy:',true_pred_ind.shape[0]/num_position_true)
print('L-1 norm:',l1norm)
print('Likelihood:', lkd)

Number of Iterations: 338
Accuracy: 0.7
L-1 norm: 0.3861762421876817
Likelihood: -4151.505524018828


# 4. Implementation of the MM algorithm

The following snippet tests the performance of the MM algorithm. Similarly, the accuracy and the L-1 norm is reported to compare the MM algorithm to the EM algorithm. We use Frank-Wolf algorithm to solve the corresponding concave optimization problem in each iteration of the MM algorithm.

In [ ]:
# Initialization of parameters
num_position = grid_size**2
w = np.random.uniform(0,1,num_position)
w = w/np.sum(w)
w = w.reshape(1,-1)
beta1 = np.repeat(-1,num_position).reshape(-1,1)
loc = cand_loc

dist1 = caldist(loc,bike_loc,bike_num)
choice_prob = np.zeros((num_position,bike_num+1,num_records))
choice_prob[:,0,:] = 1/(1+np.sum(np.exp(beta0+beta1.reshape(-1,1,1)*dist1),axis=2))
choice_prob[:,1:,:] = np.exp(beta0+beta1.reshape(-1,1,1)*np.transpose(dist1,(0,2,1)))/(1+np.sum(np.exp(beta0+beta1.reshape(-1,1,1)*dist1),axis=2)).reshape(num_position,1,-1)
diff_w = np.inf
threshold = 5e-3
iter = 0
grad_old = 0
diff_grad = np.inf
w_prime = w

# Define the function that is optimized using golden-section search method
def gold_search_func(alpha):
    f = g(w_prime+alpha*(ej-w_prime),choice_prob,book_bike,book_index)-np.sum(grad_h*(w_prime+alpha*(ej-w_prime)))
    return -f

# Frank-Wolf algorithm
while(diff_w>threshold):
    iter += 1
    gw_prime = g(w_prime,choice_prob,book_bike,book_index)
    grad_h = -num_booked *np.sum(choice_prob[:,0,:]*all_period,axis=1)/np.sum((1-np.sum(w.reshape(-1,1)*choice_prob[:,0,:],axis=0))*all_period)
    diff_grad = np.inf
    while (diff_grad>threshold):
        grad_g = find_grad_g(w_prime,choice_prob,book_bike,book_index)
        j = np.argmax(grad_g-grad_h)
        ej = np.zeros(num_position)
        ej[j] = ej[j]+1
        alpha = optimize.golden(gold_search_func, brack=(0, 1))
        w_prime = w_prime + alpha*(ej-w_prime)
        grad_new = np.abs(gw_prime-np.sum(grad_h*w_prime))
        diff_grad = np.abs(grad_new-grad_old)
        grad_old = grad_new
    diff_w = np.sum(np.abs(w_prime-w))
    w = w_prime
    

In [6]:
pre_w = w[-1]
true_w = np.zeros(grid_size**2)
true_w[true_pos_ind] = position_weight
sel_ind = np.where(pre_w>=np.sort(pre_w)[-num_position_true])[0]
true_pred_ind = np.intersect1d(sel_ind,true_pos_ind)
l1norm = np.sum(np.abs(pre_w-true_w))
lkd = findlkd_no_constraint(num_position,dist_old,
                                beta0,np.repeat(beta1_true,num_position).reshape(-1,1),pre_w,
                                bike_num,num_records,book_bike,book_index,num_booked,all_period)

print('Number of Iterations:', iter)
print('Accuracy:',true_pred_ind.shape[0]/num_position_true)
print('L1-norm:',l1norm)
print('Likelihood:',lkd)

Number of Iterations: 4
Accuracy: 0.7
L1-norm: 0.40551942443533134
Likelihood: -4151.670494402881
